In [21]:
# Modified and added comments from:
# Source: https://github.com/jonathanpaulson/AdventOfCode/blob/master/2022/17.py

# Read the file
f = open("puzzle.txt").read().strip()

# The tall, vertical chamber is exactly seven units wide. 
# Each rock appears so that its left edge is two units away 
# from the left wall and its bottom edge is three units above 
# the highest rock in the room (or the floor, if there isn't one).

def get_piece(t, y):
    '''
    Get the piece falling down. Goes in order and is determined by t % 5

    Input:
        t (int): time
        y (int): y-position of the piece
    Output:
        (set): the coordinates of the piece
    '''
    if t == 0:
        return set([(2, y), (3, y), (4, y), (5, y)])
    elif t == 1:
        return set([(3, y + 2), (2, y + 1), (3, y + 1), (4, y + 1), (3, y)])
    elif t == 2:
        return set([(2, y), (3, y), (4, y), (4, y + 1), (4, y + 2)])
    elif t == 3:
        return set([(2, y),(2, y + 1),(2, y + 2),(2, y + 3)])
    elif t == 4:
        return set([(2, y + 1),(2, y),(3, y + 1),(3, y)])


def move_left(piece):
    '''
    Move a piece left

    Input:
        piece (set): the currenet coordinates of the piece
    Output:
        (set): the updated coordinates of the piece if moved left
    '''

    # If any x-coordinate of the piece is already touching the left boundary
    # return itself
    if any([x == 0 for (x, _) in piece]):
        return piece
    
    # Default is shift the piece left
    return set([(x - 1, y) for (x, y) in piece])

def move_right(piece):
    '''
    Move a piece right

    Input:
        piece (set): the currenet coordinates of the piece
    Output:
        (set): the updated coordinates of the piece if moved right
    '''

    # If any x-coordinate of the piece is already touching the right boundary
    # return itself
    if any([x == 6 for (x, _) in piece]):
        return piece
    
    # Default is shift the piece left
    return set([(x + 1,y) for (x, y) in piece])


# Set that stores the lower boundary, initialized with the floor
R = set([(x,0) for x in range(7)])

# Keep track of current top y-coordinate
top = 0

# Keep track of ith command in f
i = 0

# Keep track of time
t = 0

# For Part 2
# Elephants want to do this 1000000000000 times
L = 1000000000000

# Empty set to keep track of rocks seen before
# Assume that the formation of rocks will also eventually repeat
SEEN = {}

# Check the number of blocks added to SEEN before the pattern occurs
added = 0

# While time is less than desired number of iterations
while t < L:

    # Get the next piece
    # top + 4 because piece spawns 3 above top
    piece = get_piece(t % 5, top + 4)

    # Loop instructions
    while True:

        # If command is left and moving left does not intersect with 
        # the left boundary or any other piece, move left
        if f[i]=='<' and (move_left(piece) & R == set()):
            piece = move_left(piece)

        # If command is right and moving right does not intersect with 
        # the right boundary or any other piece, move right
        elif f[i] == '>' and (move_right(piece) & R == set()):
            piece = move_right(piece)

        # Update ith command (restarts if EOF is reached)
        i = (i + 1) % len(f)

        # Move piece down
        piece = set([(x,y-1) for (x,y) in piece])

        # If the piece hits the floor
        # Check this by seeing if a piece has any coordinates that
        # intersect with the floor
        if piece & R:

            # Move piece up
            piece = set([(x,y+1) for (x,y) in piece])

            # Add piece to R (new floor)
            R |= piece

            # Get the new top y-coordinate
            top = max([y for (_,y) in R])


            # Part 2
            # Get top 30 rows
            # Just assumes that looking at top 30 rows will be enough to identify repeat pattern
            signature = frozenset([(x, top - y) for (x, y) in R if top - y <= 30])

            # Get current command index, piece, and current pattern
            SR = (i, t % 5, signature)

            # If this pattern has been seen and is for Part 2 only
            if SR in SEEN and t >= 2022:

                # Get the prev time and prev top this pattern was seen
                (oldt, oldy) = SEEN[SR]

                # Take the difference in top and time
                dy = top-oldy
                dt = t-oldt

                # Get the amount of times this pattern will repeat
                amt = (L - t) // dt # floor of (1 trillion - current time) divided by difference in time
                added += amt * dy # Get the amount of times that this pattern is added to the tower
                t += amt * dt # Update the time of each time this pattern happens
            
            # Add it to the SEEN set and exit
            SEEN[SR] = (t,top)
            break

    # Update time
    t += 1

    # Part 1
    if t==2022:
        print(top)

# Part 2
print(top + added)

3193
1577650429835
